In [ ]:
%matplotlib inline

import torch

import triton
import triton.language as tl

torch.cuda.set_device(4)

# GEMM 伪代码

```python
# Do in parallel
for m in range(0, M, BLOCK_SIZE_M):
  # Do in parallel
  for n in range(0, N, BLOCK_SIZE_N):
    acc = zeros((BLOCK_SIZE_M, BLOCK_SIZE_N), dtype=float32)
    for k in range(0, K, BLOCK_SIZE_K):
      a = A[m : m+BLOCK_SIZE_M, k : k+BLOCK_SIZE_K]
      b = B[k : k+BLOCK_SIZE_K, n : n+BLOCK_SIZE_N]
      acc += dot(a, b)
    C[m : m+BLOCK_SIZE_M, n : n+BLOCK_SIZE_N] = acc
```
block a 的 size 是 `BLOCK_SIZE_M * BLOCK_SIZEK`
block b 的 size 是 `BLOCK_SIZE_K * BLOCK_SIZEN`

# GEMM kernel 定义

注意 Triton 的基本操作单位是 block，我们的 kernel 需要指出每一个 block 的工作

在这里，一个 block 需要计算出矩阵 $C$ 的一个 block 最终值

让我们回顾一下 Tiling 技术，在下图中有 $A \in \mathbb{R}^{M \times K}, B \in \mathbb{R}^{K \times N}, C \in \mathbb{R}^{M \times N}$

<div>
<img src="figures/tiling.png" width="600">
</div>

以 $C_{0,0}$ 为例，有 $C_{0,0}=A_{0,0} \times B_{0,0} + A_{0,1} \times B_{1,0}$

如果把 block 划分地更多一些，那么上述的等式变为 $C_{0,0} = \sum^n_{i=0} A_{0,i} \times B_{i,0}$，其中 $n = \lceil \frac{K}{\mathtt{BLOCK\_SIZE\_K}} \rceil$

所以我们的 kernel中一定有一个循环结构，用于建模上面的 SUM 操作

In [ ]:
@triton.jit
def matmul_kernel(
    A_ptr, B_ptr, C_ptr, # pointer
    M, N, K, # size
    stride_am, stride_ak, stride_bk, stride_bn, stride_cm, stride_cn, # 矩阵两个方向的步长，以元素为单位
    BLOCK_SIZE_M: tl.constexpr, BLOCK_SIZE_N: tl.constexpr, BLOCK_SIZE_K: tl.constexpr):

    # 当前 block 计算 $C_{m, n}$
    pid_m = tl.program_id(axis=0)
    pid_n = tl.program_id(axis=1)

    offs_am = pid_m * BLOCK_SIZE_M + tl.arange(0, BLOCK_SIZE_M)
    offs_bn = pid_n * BLOCK_SIZE_N + tl.arange(0, BLOCK_SIZE_N)
    offs_k = tl.arange(0, BLOCK_SIZE_K)

    # a_ptrs 即 block a 中 BLOCK_SIZE_M * BLOCK_SIZE_K 个元素的指针
    # b_ptrs 同理
    a_ptrs = A_ptr + (offs_am[:, None] * stride_am + offs_k[None, :] * stride_ak)
    b_ptrs = B_ptr + (offs_k[:, None] * stride_bk + offs_bn[None, :] * stride_bn)

    # accumulator 为 BLOCK_SIZE_M * BLOCK_SIZE_N 的矩阵，用于建模之前提到的累加结构
    accumulator = tl.zeros((BLOCK_SIZE_M, BLOCK_SIZE_N), dtype=tl.float32)
    
    # 遍历 K 对应的维度 
    for k in range(0, tl.cdiv(K, BLOCK_SIZE_K)):
        # a 表示 $A_{m, i}$
        # b 表示 $B_{i, n}$
        a = tl.load(a_ptrs, mask=offs_k[None, :] < K - k * BLOCK_SIZE_K, other=0.0)
        b = tl.load(b_ptrs, mask=offs_k[:, None] < K - k * BLOCK_SIZE_K, other=0.0)
        accumulator += tl.dot(a, b)
        a_ptrs += BLOCK_SIZE_K * stride_ak
        b_ptrs += BLOCK_SIZE_K * stride_bk

    # 写回 $C_{m, n}$
    offs_cm = pid_m * BLOCK_SIZE_M + tl.arange(0, BLOCK_SIZE_M)
    offs_cn = pid_n * BLOCK_SIZE_N + tl.arange(0, BLOCK_SIZE_N)
    c_ptrs = C_ptr + (offs_cm[:, None] * stride_cm + offs_cn[None, :] * stride_cn)
    c_mask = (offs_cm[:, None] < M) & (offs_cn[None, :] < N)
    c = accumulator.to(tl.float16)
    tl.store(c_ptrs, c, mask=c_mask)

# kernel 的封装

矩阵是一个二维结构，所以一个比较直觉的想法就是使用一个二维的 grid 去处理，`grid` 函数需要指定 block 如何划分，`grid` 决定了 kernel 中我们调用 `tl.program_id` 的结果，在向量加法的例子中，我们的 grid 当时是这么定义的：

```python
grid = lambda meta: (triton.cdiv(n_elements, meta['BLOCK_SIZE']), )
```

这里向量是一个一维结构，所以我们返回了一个一维的 tuple；而对应地，这里矩阵乘法我们采用了二维结构。但需要注意的是，这并不是必须的，我们仍然可以把矩阵看作一个一维结构，这也是 Triton 官方 Tutorial 的做法，下面的图可以说明

<div>
<img src="figures/grid_div.png" width="600">
</div>

也就是说，`grid` 只是定义一种逻辑上的 block 划分方式，只需要保证能够正确找到矩阵 $C$ 的这个 block 对应的 $A,B$ 两个矩阵的区域即可

另外，下面的 `matmul` 中的 `BLOCK_SIZE_M` 等超参数这里是随便取的，并不一定最优

In [ ]:
def grid(META):
    return (triton.cdiv(META['M'], META['BLOCK_SIZE_M']), triton.cdiv(META['N'], META['BLOCK_SIZE_N']))

def matmul(A, B):
    assert A.shape[1] == B.shape[0]
    M, K = A.shape
    K, N = B.shape
    C = torch.empty((M, N), device='cuda', dtype=torch.float16)
    matmul_kernel[grid](
        A, B, C, 
        M, N, K, 
        A.stride(0), A.stride(1), B.stride(0), B.stride(1), C.stride(0), C.stride(1),
        BLOCK_SIZE_M=16, BLOCK_SIZE_N=16, BLOCK_SIZE_K=16)

    return C

# 单元测试

TODO：这里有一个我暂时没有解决的问题，就是如果把显卡换成 TITAN，这个计算结果会提示错误，这并不是上面的逻辑写错了，我尝试过几个 GEMM 的 Triton 实现都有这个问题，我个人猜测是在类型转换的时候，存在一定的精度误差

In [ ]:
torch.manual_seed(0)
A = torch.randn(512, 512, device='cuda', dtype=torch.float16)
B = torch.randn(512, 512, device='cuda', dtype=torch.float16)

triton_output = matmul(A, B)
pytorch_output = torch.matmul(A, B)

print(f"triton_output={triton_output}")
print(f"pytorch_output={pytorch_output}")

if torch.allclose(triton_output, pytorch_output, atol=1e-2, rtol=0):
    print("✅ Triton and Torch match")
else:
    print("❌ Triton and Torch differ")